In [145]:
import os
import sys

curr_dir = os.getcwd()
src_dir = os.path.join(os.path.dirname(curr_dir), "src")
sql_dir = os.path.join(os.path.dirname(curr_dir), "sql")
sys.path.append(src_dir)
sys.path.append(sql_dir)

In [146]:
import json
import warnings
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from cleaning_helper import print_steam_links, check_na
from db import get_db
from settings import Path
from sqlalchemy import text

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)

### Data Extraction from SQL Database

In [147]:
def fetch_data(source: str):
    """
    Fetches data from a specified source and returns it as a pandas DataFrame.

    Parameters:
    source (str): The name of the source file containing the SQL query.

    Returns:
    pandas.DataFrame: The fetched data as a DataFrame.

    """
    db = get_db()

    with open(os.path.join(sql_dir, source), "r") as f:
        query = text(f.read())
    result = db.execute(query)
    data = result.fetchall()
    columns = result.keys()
    df = pd.DataFrame(data, columns=columns)

    db.close()

    return df

steam_data = fetch_data("get_all_steam_data.sql")

In [148]:
steam_data.head()

,type,name,appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,supported_languages,reviews,header_image,capsule_image,website,requirements,developers,publishers,price_overview,platform,metacritic,categories,genres,recommendations,achievements,release_date,coming_soon
0,game,Counter-Strike,10,0,0,None,[],Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English\n*\n, French\n*\n, German\n*\n, Italia...",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": ""\r\n\t\t\t<p><strong>Minimum:</st...","[""Valve""]","[""Valve""]","{""final"": 99, ""initial"": 999, ""currency"": ""USD...","{""mac"": true, ""linux"": true, ""windows"": true}",88,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}]",150423,0,"Nov 1, 2000",0
1,game,Team Fortress Classic,20,0,0,None,[],One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": ""\r\n\t\t\t<p><strong>Minimum:</st...","[""Valve""]","[""Valve""]","{""final"": 74, ""initial"": 499, ""currency"": ""USD...","{""mac"": true, ""linux"": true, ""windows"": true}",0,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}]",6133,0,"Apr 1, 1999",0
2,game,Day of Defeat,30,0,0,None,[],Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,http://www.dayofdefeat.com/,"{""minimum"": ""\r\n\t\t\t<p><strong>Minimum:</st...","[""Valve""]","[""Valve""]","{""final"": 74, ""initial"": 499, ""currency"": ""USD...","{""mac"": true, ""linux"": true, ""windows"": true}",79,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}]",4074,0,"May 1, 2003",0
3,game,Deathmatch Classic,40,0,0,None,[],Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": ""\r\n\t\t\t<p><strong>Minimum:</st...","[""Valve""]","[""Valve""]","{""final"": 74, ""initial"": 499, ""currency"": ""USD...","{""mac"": true, ""linux"": true, ""windows"": true}",0,"[{""id"": 1, ""description"": ""Multi-player""}, {""i...","[{""id"": ""1"", ""description"": ""Action""}]",2149,0,"Jun 1, 2001",0
4,game,Half-Life: Opposing Force,50,0,0,None,[],Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,,"{""minimum"": ""\r\n\t\t\t<p><strong>Minimum:</st...","[""Gearbox Software""]","[""Valve""]","{""final"": 74, ""initial"": 499, ""currency"": ""USD...","{""mac"": true, ""linux"": true, ""windows"": true}",0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""1"", ""description"": ""Action""}]",19244,0,"Nov 1, 1999",0


Creating a copy of `steam_data` dataset before starting the cleaning process.

In [149]:
raw_steam_data = steam_data.copy()

### Process Null values

Since the data is queried from SQL, some null vales are read as strings. 

In [150]:
def process_null(df):
    """
    Process null values in a DataFrame by replacing specific values with None.

    Args:
        df (pandas.DataFrame): The DataFrame to process.

    Returns:
        pandas.DataFrame: The processed DataFrame with null values replaced.

    """
    df = df.copy()

    convert_to_none = ['', 'none', 'null', 'N/a', 'n/a', '["none"]', '["null"]']
    df.replace(convert_to_none, None, inplace=True)
    
    return df

raw_steam_data = process_null(raw_steam_data)

In [151]:
raw_steam_data.isnull().sum()

type                        0
name                       15
appid                       0
required_age                0
is_free                     0
controller_support      57146
dlc                         0
detailed_description      121
about_the_game            146
short_description          64
supported_languages        82
reviews                 63577
header_image                0
capsule_image               0
website                 36322
requirements                0
developers                232
publishers                 11
price_overview           9885
platform                    0
metacritic                  0
categories                801
genres                    133
recommendations             0
achievements                0
release_date              112
coming_soon                 0
dtype: int64

### Processing Age

In [152]:
raw_steam_data['required_age'].value_counts()

required_age
0     72901
17      606
18      155
13      121
16       50
12       35
10       14
15       14
7         9
6         8
3         5
14        4
11        2
21        1
5         1
Name: count, dtype: int64

Reducing the number of categories that ages fall into by following the [PEGI age ratings](https://pegi.info) categories. For example, instead of comparing games rated as 5, 6, 7 or 8, we could compare games rated 5+ or 8+. 

In [153]:
def process_age(df):
    """
    Process the 'required_age' column in the given DataFrame.

    Parameters:
    df (DataFrame): The input DataFrame containing the 'required_age' column.

    Returns:
    DataFrame: A copy of the input DataFrame with the 'required_age' column processed.

    """
    df = df.copy()
    cut_points = [-1, 0, 3, 7, 12, 16, 1000]
    categories = [0, 3, 7, 12, 16, 18]

    df = df[df['required_age'].notna()] 
    df['required_age'] = pd.cut(df['required_age'], bins=cut_points, labels=categories)
    
    return df

age_df = process_age(raw_steam_data)
age_df['required_age'].value_counts().sort_index()

required_age
0     72901
3         5
7        18
12       51
16      189
18      762
Name: count, dtype: int64

### Processing Platforms

In [154]:
platforms_first_row = age_df['platform'].iloc[0]
print(type(platforms_first_row))
platforms_first_row

<class 'str'>


'{"mac": true, "linux": true, "windows": true}'

First task is to recognise the data as dictionaries rather than strings.

In [155]:
eval_first_row = json.loads(platforms_first_row)
print(type(eval_first_row))
print(eval_first_row)
eval_first_row['windows']

<class 'dict'>
{'mac': True, 'linux': True, 'windows': True}


True

Formatting the output. Let's keep things simple and return a string of supported platforms

In [156]:
';'.join(eval_first_row.keys())

'mac;linux;windows'

Keeping only keys that have `True` value

In [157]:
platforms = {'windows': True, 'mac': True, 'linux': False}
print([x for x in platforms.keys() if platforms[x]])
';'.join(x for x in platforms.keys() if platforms[x])

['windows', 'mac']


'windows;mac'

Creating a funtion by using pandas [Series.apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html)

In [158]:
def process_platforms(df):
    """
    Process the platforms column in the given DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing the platforms column.

    Returns:
        pandas.DataFrame: The DataFrame with the platforms column processed.
    """
    df = df.copy()

    def parse_platforms(x):
        """
        Parse the platforms from the given JSON string.

        Args:
            x (str): The JSON string representing the platforms.

        Returns:
            str: The parsed platforms separated by semicolons.
        """
        d = json.loads(x)

        return ';'.join(platform for platform in d.keys() if d[platform])

    df['platform'] = df['platform'].apply(parse_platforms)

    return df

platforms_df = process_platforms(age_df)
platforms_df['platform'].value_counts()

platform
windows              55926
mac;linux;windows     8065
mac;windows           7448
linux;windows         2464
mac                     11
linux                   11
mac;linux                1
Name: count, dtype: int64

### Processing Languages

In [159]:
platforms_df['supported_languages'].value_counts().head(10)

supported_languages
English                                                           19780
English\n*\n*\nlanguages with full audio support                  17326
English, Russian                                                   1386
English, Simplified Chinese                                        1020
English, Japanese                                                   888
Simplified Chinese                                                  827
Simplified Chinese\n*\n*\nlanguages with full audio support         649
English\n*\n, Russian\n*\n*\nlanguages with full audio support      457
English, Portuguese - Brazil                                        421
English, French                                                     394
Name: count, dtype: int64

English plus other languages make up most of the data. It seems reasonably safe to assume that if the app is in English, the word English will appear somewhere in this string. Use the Series.apply to check if the string 'english' appears in each row. Null values are dropped as they are very less in number.

In [160]:
def process_language(df):
    """
    Process the language data in the given DataFrame.

    Parameters:
    df (DataFrame): The input DataFrame containing the language data.

    Returns:
    DataFrame: The processed DataFrame with the language data cleaned.

    """
    df = df.copy()
    
    df = df.dropna(subset=['supported_languages'])
    
    df['english'] = df['supported_languages'].apply(lambda x: 1 if 'english' in x.lower() else 0)
    df = df.drop('supported_languages', axis=1)
    
    return df

language_df = process_language(platforms_df)
language_df[['name', 'english']].head()

,name,english
0,Counter-Strike,1
1,Team Fortress Classic,1
2,Day of Defeat,1
3,Deathmatch Classic,1
4,Half-Life: Opposing Force,1


In [161]:
language_df['english'].value_counts()

english
1    71127
0     2717
Name: count, dtype: int64

### Processing Developers and Publishers

In [162]:
print('Developers null counts:', language_df[language_df['developers'].isnull()].shape[0])
print('Developers empty list counts:', language_df[language_df['developers'] == '[""]'].shape[0])

print('\nPublishers null counts:', language_df['publishers'].isnull().sum())
print('Publishers empty list counts:', language_df[language_df['publishers'] == '[""]'].shape[0])

Developers null counts: 214
Developers empty list counts: 0

Publishers null counts: 10
Publishers empty list counts: 503


A few options for dealing with these two columns:

- Remove all rows missing either developer or publisher information
- Impute missing information by replacing the missing column with the column we have (i.e. if developers is missing, fill it with the value in publishers)
- Fill missing information with 'Unknown' or 'None'

In [163]:
no_dev = language_df[language_df['developers'].isnull()]
print('Total games missing developer:', no_dev.shape[0], '\n')
print_steam_links(no_dev[:5])

Total games missing developer: 214 

Crash Time 2: http://store.steampowered.com/app/11390
18 Wheels of Steel: Extreme Trucker: http://store.steampowered.com/app/33730
Prison Tycoon 4: SuperMax: http://store.steampowered.com/app/33750
Jewel Quest Pack: http://store.steampowered.com/app/37960
Mahjong Quest Collection: http://store.steampowered.com/app/38000


In [164]:
no_pub = language_df[language_df['publishers'] == '[""]']
print('\nTotal games missing publisher:', no_pub.shape[0], '\n')
print_steam_links(no_pub[:5])


Total games missing publisher: 503 

RIP - Trilogy™: http://store.steampowered.com/app/2540
Vigil: Blood Bitterness™: http://store.steampowered.com/app/2570
ThreadSpace: Hyperbol: http://store.steampowered.com/app/2720
Bullet Candy: http://store.steampowered.com/app/6600
Loki: http://store.steampowered.com/app/7260


In [165]:
no_dev_or_pub = language_df[(language_df['developers'].isnull()) & (language_df['publishers'] == '[""]')]
print('\nTotal games missing developer and publisher:', no_dev_or_pub.shape[0], '\n')
print_steam_links(no_dev_or_pub[:5])


Total games missing developer and publisher: 137 

Guardians of Graxia: http://store.steampowered.com/app/90500
Patterns: http://store.steampowered.com/app/218980
PlayClaw 5 - Game Recording and Streaming: http://store.steampowered.com/app/237370
Artemis Spaceship Bridge Simulator: http://store.steampowered.com/app/247350
A Walk in the Dark: http://store.steampowered.com/app/248730


Some titles may have been self-publishes while others simply have wrong ot missing data. As the priority is creating a clean dataset, and there are only a few hundred rows, it will be fine to remove them from the data. Is is also safe to assume that multiple developers or publishers may have been involved in the game.

In [166]:
def process_developers_and_publishers(df):
    """
    Process the 'developers' and 'publishers' columns in the given DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing the 'developers' and 'publishers' columns.

    Returns:
        None

    This function filters out rows where the 'developers' column is not equal to None and the 'publishers'
    column is not equal to '[""]'. It then converts the values in the 'developers' and 'publishers' columns
    from strings to lists using the `literal_eval` function from the `ast` module. Finally, it prints the number
    of rows in each column that have multiple values.

    Note:
        This function modifies the input DataFrame in-place.
    """
    df = df[(df['developers'].notna()) & ((df['publishers'].notna())) & (df['publishers'] != '[""]')].copy()
    
    for col in ['developers', 'publishers']:
        df[col] = df[col].apply(lambda x: literal_eval(x))
        
        num_rows = df[df[col].str.len() > 1].shape[0]
        
        print('Rows in {} column with multiple values:'.format(col), num_rows)

process_developers_and_publishers(language_df)

Rows in developers column with multiple values: 4883
Rows in publishers column with multiple values: 2560


The column can be formateed by joining the multiple developers and publishers into a string. Since a number of developers and publishers have a comma in their name (for example PopCap Games, Inc.), they can be joint on a semi-colon. There are some that have `[NA]` or `[N/A]` or `[N/a]`.

In [167]:
# def process_developers_and_publishers(df):
#     """
#     Process the developers and publishers columns in the given DataFrame.

#     Args:
#         df (pandas.DataFrame): The DataFrame containing the 'developers' and 'publishers' columns.

#     Returns:
#         pandas.DataFrame: The DataFrame with the 'developers' and 'publishers' columns processed and updated.
#     """
#     df = df[(df['developers'].notna()) & (df['publishers'] != '[""]')].copy()
#     df = df[~(df['developers'].str.contains(';')) & ~(df['publishers'].str.contains(';'))]
#     df = df[(df['publishers'] != '["NA"]') & (df['publishers'] != '["N/A"]') & (df['publishers'] != '["N/a"]') & (df['developers'] != '["NA"]') & (df['developers'] != '["N/A"]') & (df['developers'] != '["N/a"]')]
    
#     df['developer'] = df['developers'].apply(lambda x: ';'.join(literal_eval(x)))
#     df['publisher'] = df['publishers'].apply(lambda x: ';'.join(literal_eval(x)))

#     df = df.drop(['developers', 'publishers'], axis=1)
    
#     return df

# dev_pub_df = process_developers_and_publishers(language_df)
# dev_pub_df[['name', 'appid', 'developer', 'publisher']].head()

def process_developers_and_publishers(df):
    """
    Process the developers and publishers columns in the given DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing the developers and publishers columns.

    Returns:
        pandas.DataFrame: The processed DataFrame with the developers and publishers columns transformed.

    """
    pattern = r'(?i)\["(n/a|na|null)"\]'

    df = df[(df['developers'].notna()) & (df['publishers'].notna() & (df['publishers'] != '[""]'))]
    df = df[~df['developers'].str.contains(pattern, na=False)]
    df = df[~df['publishers'].str.contains(pattern, na=False)]

    df = df[~df['developers'].str.contains(';', na=False)]
    df = df[~df['publishers'].str.contains(';', na=False)]

    def safe_literal_eval(val):
        try:
            result = literal_eval(val)
            if isinstance(result, list):
                return ';'.join(filter(None, result))
        except (ValueError, SyntaxError):
            return ''
        return val
    
    df['developer'] = df['developers'].apply(safe_literal_eval)
    df['publisher'] = df['publishers'].apply(safe_literal_eval)

    df = df.drop(['developers', 'publishers'], axis=1)
    
    return df

dev_pub_df = process_developers_and_publishers(language_df)
dev_pub_df[['name', 'appid', 'developer', 'publisher']].head()


,name,appid,developer,publisher
0,Counter-Strike,10,Valve,Valve
1,Team Fortress Classic,20,Valve,Valve
2,Day of Defeat,30,Valve,Valve
3,Deathmatch Classic,40,Valve,Valve
4,Half-Life: Opposing Force,50,Gearbox Software,Valve


Helper cell to check if any values of `[n/a]` variations were missed.

In [168]:
# temp = check_na(dev_pub_df, 'publisher')
# temp.head()

### Processing Price

In [169]:
dev_pub_df[dev_pub_df['price_overview'].isnull()].shape[0]

9689

In [170]:
free_and_null_price = dev_pub_df[(dev_pub_df['is_free']) & (dev_pub_df['price_overview'].isnull())]
print("Games that are marked free and have price as null: ", free_and_null_price.shape[0])

Games that are marked free and have price as null:  6036


It turns out this accounts for most of the missing values in the `price_overview` column. These missing values can be handled by setting the final price as 0. This makes intuitive sense - free games wouldn't have a price.

In [171]:
not_free_and_null_price = dev_pub_df[(dev_pub_df['is_free'] == 0) & (dev_pub_df['price_overview'].isnull())]
print("Steam store links to some games that are not free and have missing price\n")
print_steam_links(not_free_and_null_price[:5])

Steam store links to some games that are not free and have missing price

Half-Life 2: Lost Coast: http://store.steampowered.com/app/340
Final DOOM: http://store.steampowered.com/app/2290
Quake II Mission Pack: The Reckoning: http://store.steampowered.com/app/2330
Quake II Mission Pack: Ground Zero: http://store.steampowered.com/app/2340
The Ship: Single Player: http://store.steampowered.com/app/2420


Some of these games are sold as part of the main game, some have been removed while others have their name replaced.

In [172]:
dev_pub_df['price_overview'][37]

'{"final": 119, "initial": 599, "currency": "USD", "final_formatted": "$1.19", "discount_percent": 80, "initial_formatted": "$5.99"}'

This will be formatted by taking only the `currency` and `initial` keys. These will be created as two seperate columns. There are multiple currencies and it makes sense to convert these to USD.

In [173]:
def process_price(df):
    """
    Process the price data in the given DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing the price data.

    Returns:
        pandas.DataFrame: The DataFrame with processed price data.
    """
    df = df.copy()
    currency_rates = {'EUR': 1.08, 'TWD': 0.03, 'SGD': 0.74, 'BRL': 0.18, 'AUD': 0.67} 
        
    def parse_price(x):
        if x is not None:
            return literal_eval(x)
        else:
            return {'currency': 'USD', 'initial': -1}
    
    def convert_to_usd(price, currency, rates):
        if currency == 'USD':
            return price
        
        currency_rate = rates[currency]
        return price * currency_rate
    
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    df.loc[df['is_free'] == 1, 'price'] = 0
    df.loc[df['price'] > 0, 'price'] /= 100

    df['price'] = df.apply(lambda x: convert_to_usd(x['price'], x['currency'], currency_rates), axis=1)

    df = df.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    return df

price_df = process_price(dev_pub_df)
price_df[['name', 'price']].head()

,name,price
0,Counter-Strike,9.99
1,Team Fortress Classic,4.99
2,Day of Defeat,4.99
3,Deathmatch Classic,4.99
4,Half-Life: Opposing Force,4.99


### Processing Categories and Genres

In [174]:
print("Number of null values for categories: ", price_df[price_df['categories'].isnull()].shape[0])
print("Number of null values for genres: ", price_df[price_df['genres'].isnull()].shape[0])

Number of null values for categories:  797
Number of null values for genres:  99


Randomly inspect null rows by using [Dataframe.sample](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) method.

In [175]:
print_steam_links(price_df[price_df['categories'].isnull()].sample(5, random_state=0))

RPG Paper Maker: http://store.steampowered.com/app/1066860
PhotoTangler Collage Maker: http://store.steampowered.com/app/586560
MasterPlan: http://store.steampowered.com/app/1269310
Home Office Tasker: http://store.steampowered.com/app/1727670
Hypnosia - Application d'Hypnose avec Reconnaissance Vocale: http://store.steampowered.com/app/2952000


All of the above rows of categories are applications or software of some kind, and not actually games. It would be best to remove these.

In [176]:
print_steam_links(price_df[price_df['genres'].isnull()].sample(5, random_state=0))

ChessBase 13 Academy: http://store.steampowered.com/app/377340
General Staff: Black Powder Battle Designer Bundle: http://store.steampowered.com/app/2231050
Call of Cthulhu®: Dark Corners of the Earth: http://store.steampowered.com/app/22340
River Relaxation VR: http://store.steampowered.com/app/938760
Evil Shogun: http://store.steampowered.com/app/1505830


Most of these are games and suggests that genre data simply wasn't supplied. These rows can be removed as these are less in number.

In [177]:
def process_categories_and_genres(df):
    """
    Process the categories and genres columns of the given DataFrame.
    
    Parameters:
    df (DataFrame): The input DataFrame containing the 'categories' and 'genres' columns.
    
    Returns:
    DataFrame: The processed DataFrame with the 'categories' and 'genres' columns modified.
    """
    df = df.copy()
    df = df[(df['categories'].notna()) & (df['genres'].notna())]
    
    for col in ['categories', 'genres']:
        df[col] = df[col].apply(lambda x: ';'.join(item['description'] for item in literal_eval(x)))
    
    return df

cat_gen_df = process_categories_and_genres(price_df)
cat_gen_df[['appid', 'categories', 'genres']].head()

,appid,categories,genres
0,10,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
1,20,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
2,30,Multi-player;Valve Anti-Cheat enabled;Family S...,Action
3,40,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
4,50,Single-player;Multi-player;Valve Anti-Cheat en...,Action


### Processing Game Descriptions

In [180]:
cat_gen_df[['detailed_description', 'about_the_game', 'short_description']].isnull().sum()

detailed_description     81
about_the_game          106
short_description        24
dtype: int64

In [183]:
cat_gen_df[(cat_gen_df['detailed_description'].isnull()) & (cat_gen_df['about_the_game'].isnull()) & (cat_gen_df['short_description'].isnull())].head()

,type,name,appid,required_age,controller_support,dlc,detailed_description,about_the_game,short_description,reviews,header_image,capsule_image,website,requirements,platform,metacritic,categories,genres,recommendations,achievements,release_date,coming_soon,english,developer,publisher,price
11716,game,Gamepad Controller Template,523080,0,None,[],None,None,None,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,None,"{""minimum"": ""<strong>Minimum:</strong><br><ul ...",mac;windows,0,Single-player;Multi-player;PvP;Shared/Split Sc...,Action;Casual;Indie;Sports;Early Access,0,0,"1 Sep, 2016",0,1,K,K,5.34
25809,game,Tranquil Garden,891520,0,None,[],None,None,None,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,https://glassrobotgames.com/tranquil-garden,"{""minimum"": ""<strong>Minimum:</strong><br><ul ...",windows,0,Single-player;Partial Controller Support;Famil...,Adventure;Casual;Indie;RPG;Strategy,0,0,"Sep 18, 2020",0,1,Glass Robot,"VoxPop Games, Inc.",-1.00
34299,game,CLOSED,1131330,18,None,[],None,None,None,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,None,"{""minimum"": ""<strong>Minimum:</strong><br><ul ...",windows,0,Single-player;Partial Controller Support;Steam...,Action;Indie;RPG,0,0,"Sep 3, 2019",0,1,Kazuya;tensun3d,赤いトンボ,-1.00
35292,game,Lif,1157360,0,None,[],None,None,None,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,None,"{""minimum"": ""<strong>Minimum:</strong><br><ul ...",linux,0,Single-player;Family Sharing,Action;Indie;RPG;Sports,0,0,"Sep 27, 2019",0,1,MZ,MZ,-1.00
35974,game,Math The Question,1176440,0,None,[],None,None,None,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,None,"{""minimum"": ""<strong>Minimum:</strong><br><ul ...",windows,0,Single-player;Family Sharing,Casual;Indie,0,0,"Oct 24, 2019",0,1,K'sG,K'sG,-1.00


In [189]:
for i in cat_gen_df.columns:
    print(i, cat_gen_df[cat_gen_df[i]=='none'].shape[0])

type 0
name 0
appid 0
required_age 0
controller_support 0
dlc 0
detailed_description 0
about_the_game 0
short_description 0
reviews 0
header_image 0
capsule_image 0
website 0
requirements 0
platform 0
metacritic 0
categories 0
genres 0
recommendations 0
achievements 0
release_date 0
coming_soon 0
english 0
developer 0
publisher 0
price 0


In [184]:
raw_steam_data[raw_steam_data['appid']==745840]

,type,name,appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,supported_languages,reviews,header_image,capsule_image,website,requirements,developers,publishers,price_overview,platform,metacritic,categories,genres,recommendations,achievements,release_date,coming_soon
20052,game,Cut 2017,745840,0,0,None,[],Casual games with a little bit of horror.,Casual games with a little bit of horror.,Casual games with a little bit of horror.,Simplified Chinese,None,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,None,"{""minimum"": ""<strong>Minimum:</strong><br><ul ...","[""Mianwowo""]","[""Mianwowo""]","{""final"": 599, ""initial"": 599, ""currency"": ""US...","{""mac"": true, ""linux"": false, ""windows"": true}",0,"[{""id"": 2, ""description"": ""Single-player""}, {""...","[{""id"": ""25"", ""description"": ""Adventure""}, {""i...",0,11,"Nov 29, 2017",0


In [186]:
cat_gen_df.isnull().sum()

type                        0
name                        0
appid                       0
required_age                0
controller_support      55668
dlc                         0
detailed_description       81
about_the_game            106
short_description          24
reviews                 62167
header_image                0
capsule_image               0
website                 35539
requirements                0
platform                    0
metacritic                  0
categories                  0
genres                      0
recommendations             0
achievements                0
release_date               89
coming_soon                 0
english                     0
developer                   0
publisher                   0
price                       0
dtype: int64

In [187]:
cat_gen_df[cat_gen_df['release_date'].isna() & cat_gen_df['coming_soon'] == 1].shape

(19, 26)

In [193]:
cat_gen_df[cat_gen_df['requirements']=="{}"].shape

(27, 26)